In [1]:
"""Variational auto-encoder for MNIST data.

References
----------
http://edwardlib.org/tutorials/decoder
http://edwardlib.org/tutorials/inference-networks
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import os
import tensorflow as tf

from edward.models import Bernoulli, Normal
from edward.util import Progbar
from keras.layers import Dense
from observations import mnist
from scipy.misc import imsave


ed.set_seed(42)

Using TensorFlow backend.


In [2]:
def generator(array, batch_size):
  """Generate batch with respect to array's first axis."""
  start = 0  # pointer to where we are in iteration
  while True:
    stop = start + batch_size
    diff = stop - array.shape[0]
    if diff <= 0:
      batch = array[start:stop]
      start += batch_size
    else:
      batch = np.concatenate((array[start:], array[:diff]))
      start = diff
    batch = batch.astype(np.float32) / 255.0  # normalize pixel intensities
    batch = np.random.binomial(1, batch)  # binarize images
    yield batch

    
data_dir = "/tmp/data"
out_dir = "/tmp/out"
if not os.path.exists(out_dir):
  os.makedirs(out_dir)
M = 100  # batch size during training
d = 2  # latent dimension

# DATA. MNIST batches are fed at training time.
(x_train, y_train), (x_test, y_test) = mnist(data_dir)
x_train_generator = generator(x_train, M)

(x_train.shape, x_test.shape)

((60000, 784), (10000, 784))

In [5]:
# MODEL
# Define a subgraph of the full model, corresponding to a minibatch of
# size M.
z = Normal(loc=tf.zeros([M, d]), scale=tf.ones([M, d]))
hidden = Dense(256, activation='relu')(z.value())
x = Bernoulli(logits=Dense(28 * 28)(hidden))

# INFERENCE
# Define a subgraph of the variational model, corresponding to a
# minibatch of size M.
x_ph = tf.placeholder(tf.int32, (None, 28 * 28))
hidden = Dense(256, activation='relu')(tf.cast(x_ph, tf.float32))
qz = Normal(loc=Dense(d)(hidden),
            scale=Dense(d, activation='softplus')(hidden))

# Bind p(x, z) and q(z | x) to the same TensorFlow placeholder for x.
inference = ed.KLqp({z: qz}, data={x: x_ph})
optimizer = tf.train.RMSPropOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer)

tf.global_variables_initializer().run()

inference, optimizer

(<edward.inferences.klqp.KLqp at 0x7f5671dc7fd0>,
 <tensorflow.python.training.rmsprop.RMSPropOptimizer at 0x7f5671d43208>)

In [10]:
n_epoch = 5
n_iter_per_epoch = x_train.shape[0] // M
for epoch in range(1, n_epoch + 1):
  print("Epoch: {0}".format(epoch))
  avg_loss = 0.0

  pbar = Progbar(n_iter_per_epoch)
  for t in range(1, n_iter_per_epoch + 1):
    pbar.update(t)
    x_batch = next(x_train_generator)
    info_dict = inference.update(feed_dict={x_ph: x_batch})
    avg_loss += info_dict['loss']

  # Print a lower bound to the average marginal likelihood for an
  # image.
  avg_loss = avg_loss / n_iter_per_epoch
  avg_loss = avg_loss / M
  print("-log p(x) <= {:0.3f}".format(avg_loss))

  # Prior predictive check.
  images = x.eval()
  for m in range(M):
    imsave(os.path.join(out_dir, '%d.png') % m, images[m].reshape(28, 28))
    imsave(os.path.join(out_dir, '%d-in.png') % m, x_batch[m].reshape(28, 28))

Epoch: 1
600/600 [100%] ██████████████████████████████ Elapsed: 4s
-log p(x) <= 159.896
Epoch: 2
600/600 [100%] ██████████████████████████████ Elapsed: 4s
-log p(x) <= 159.821
Epoch: 3
600/600 [100%] ██████████████████████████████ Elapsed: 4s
-log p(x) <= 159.698
Epoch: 4
600/600 [100%] ██████████████████████████████ Elapsed: 4s
-log p(x) <= 159.624
Epoch: 5
600/600 [100%] ██████████████████████████████ Elapsed: 4s
-log p(x) <= 159.612


In [15]:
qz

<ed.RandomVariable 'Normal_3/' shape=(?, 2) dtype=float32>